In [1]:
import numpy as np
import pandas as pd

### Load data

In [2]:
dna_data = pd.read_csv('data/dna_seq_sample.txt', header=None, names=['dna_sequence'])
dna_data

,dna_sequence
0,GGTCAGAAAAAGCCCTCTCCATGTCTACTCACGATACATCCCTGAA...


In [3]:
len(dna_data['dna_sequence'][0])

1157

### Test example

In [4]:
def DNA_corresponding_PROTEIN(x):
    table = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
    'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    outcome = ""
    for i in range(0,len(x)-(len(x)%3),3):
        a = table[x[i:i+3]]
        outcome = outcome + a
    outcome = outcome.replace("_","")
    return outcome

In [5]:
DNA_corresponding_PROTEIN(dna_data['dna_sequence'][0])

'GQKKPSPCLLTIHPKPLRKWLFRSSCFASLGLGLLPMYFSLSIISLQSRLVLNRGPDKFDTWLWPMPLSSSLYFQTTLLLQLFLKLTSNVNNSSLASWQEAQTCVQLVFVSISLSHLFLLIQVKEYSEQVSQTWQVILVTVVGSSVSITSTFQLRSLVHSQTITITLKASCSVPLLISVALSSGLPMMPHSASWSGPVSPWYFSSIDIVRECSTYSLSIRTPGAKQRPQQPILSCWSHLLAFIFVLFVSSFTPILYILIIPGIAMTFWFRVSLQFLLYCPSETLRVLVLCSSTVESQSHKCQTQKTALLIPLNTLFHKYVFKSLYEQGMVLTAILIKEGYNHLLIKDFWLESDNSELFTTLHS'

## Eucaryotes Cell

In [6]:
from src.protein_synthesis import EucaryotesCell

### Transcription

In [7]:
cell = EucaryotesCell(dna_data['dna_sequence'][0])
cell.synthesize_protein()

In [8]:
cell.get_dna()[:30]

'GGTCAGAAAAAGCCCTCTCCATGTCTACTC'

In [9]:
cell.get_mrna()

'CH3GPPP-AGAGUAGUAAGGGACUCCGUAACGUUACUGUAAAACCAAAGCCCAAAGGGAUGUUAAAGAGGAAA-AAAA'

In [10]:
len(cell.get_mrna())

78

### Translation